# Růst a konvergence

## Cíl analýzy

- Zlepšuje se situace České republiky v rámci Evropské unie z pohledu:
  - Růstu GDP
  - Růstu mezd
  - Růstu příjmů domácností

Cílem analýzy je zjistit, zda a kdy doženeme vybrané státy EU na základě průměrných růstů hodnot v letech 2010-2019.

## Data & Metoda

Vybraná data pocházejí z databáze Eurostatu:
- GDP na obyvatele
- Mzdové náklady
- Příjmy domácností

In [58]:
import eurostat
from scipy.stats.mstats import gmean
import pandas
import numpy

LOCATIONS = ["CZ", "DE", "AT", "EU27_2020", "ES"]
BASE_LOCATION = "CZ"
START_YEAR = 2010
END_YEAR = 2019

# G1GQ = "Gross domestic product at market prices"
# D11 = "Wages and salaries"

data_1 = eurostat.get_data_df('nama_10_pc')
data_1 = data_1[data_1["na_item"].isin(["B1GQ", "D11"])]
data_1 = data_1[data_1["geo\\time"].isin(LOCATIONS)]
data_1 = data_1[data_1["unit"].isin(["CP_PPS_HAB", "CP_EUR_HAB"])]
data_1["na_item"] = data_1["na_item"] + data_1["unit"]
data_1.head()

,unit,na_item,geo\time,2020,2019,2018,2017,2016,2015,2014,...,1984,1983,1982,1981,1980,1979,1978,1977,1976,1975
1481,CP_EUR_HAB,B1GQCP_EUR_HAB,AT,42540.0,44780.0,43610.0,42000.0,40920.0,39890.0,38990.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1486,CP_EUR_HAB,B1GQCP_EUR_HAB,CZ,20120.0,21140.0,19850.0,18330.0,16790.0,16080.0,15000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1487,CP_EUR_HAB,B1GQCP_EUR_HAB,DE,40490.0,41800.0,40620.0,39530.0,38070.0,37050.0,36150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1494,CP_EUR_HAB,B1GQCP_EUR_HAB,ES,23690.0,26420.0,25750.0,24970.0,23980.0,23220.0,22220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1496,CP_EUR_HAB,B1GQCP_EUR_HAB,EU27_2020,29890.0,31310.0,30290.0,29320.0,28200.0,27500.0,26580.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
data_2 = eurostat.get_data_df('earn_nt_net')
data_2 = data_2[data_2["ecase"] == "CPL_CH2_AW100"]
data_2 = data_2[data_2["estruct"] == "NET"]
data_2 = data_2[data_2["currency"] == "EUR"]
data_2 = data_2[data_2["geo\\time"].isin(LOCATIONS)]
data_2["na_item"] = "EARN_NT_NET"
data_2["unit"] = "EUR"
data_2.head()

,currency,estruct,ecase,geo\time,2020,2019,2018,2017,2016,2015,...,2007,2006,2005,2004,2003,2002,2001,2000,na_item,unit
988,EUR,NET,CPL_CH2_AW100,AT,42367.09,41162.12,37847.53,37220.14,36707.82,34414.48,...,29156.03,28577.26,27969.56,27190.26,26837.62,25885.11,25475.92,25227.80,EARN_NT_NET,EUR
993,EUR,NET,CPL_CH2_AW100,CZ,15044.67,15523.25,14901.52,13460.59,12400.77,11526.74,...,9589.03,8238.63,7250.98,6290.27,6052.39,6159.65,5224.26,4761.09,EARN_NT_NET,EUR
994,EUR,NET,CPL_CH2_AW100,DE,41906.20,40947.00,39677.88,38506.23,37908.73,37098.03,...,31055.10,30305.54,30175.28,29920.15,28872.00,28613.40,27674.96,26831.60,EARN_NT_NET,EUR
999,EUR,NET,CPL_CH2_AW100,ES,23135.71,23386.32,23117.99,22857.77,22766.68,22785.53,...,19214.65,18344.53,17944.65,17531.09,17060.87,16236.28,15751.25,15324.04,EARN_NT_NET,EUR
1001,EUR,NET,CPL_CH2_AW100,EU27_2020,29122.58,28475.11,27561.29,26934.95,26467.96,25852.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EARN_NT_NET,EUR


In [60]:
data = pandas.concat([data_1, data_2])

### Průměrný růst ukazatelů v letech 2010 až 2019

In [61]:
data = pandas.melt(data, id_vars=["unit", "na_item", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)))
data_last_year = data[data["variable"] == 2019]
data = data.sort_values(["na_item", "geo\\time", "variable"])
data["change"] = data.groupby(["na_item", "geo\\time"])["value"].pct_change() + 1
data = data.dropna()
data = data.pivot_table(index="na_item", columns="geo\\time", values="change", aggfunc=gmean)
data = data - 1
data

geo\time,AT,CZ,DE,ES,EU27_2020
na_item,,,,,
B1GQCP_EUR_HAB,0.026493,0.038706,0.030344,0.015326,0.025779
B1GQCP_PPS_HAB,0.025245,0.038016,0.027099,0.019879,0.026954
EARN_NT_NET,0.029807,0.028891,0.022362,0.010648,0.019003


### Hodnoty ukazatelů v roce 2019 v EUR

In [62]:
data_last_year = data_last_year[["na_item", "geo\\time", "value"]]
data_last_year = data_last_year.pivot(index="na_item", columns="geo\\time", values="value")
data_last_year

geo\time,AT,CZ,DE,ES,EU27_2020
na_item,,,,,
B1GQCP_EUR_HAB,44780.00,21140.00,41800.0,26420.00,31310.00
B1GQCP_PPS_HAB,40250.00,29730.00,38620.0,28950.00,31990.00
EARN_NT_NET,41162.12,15523.25,40947.0,23386.32,28475.11


### Za kolik let ČR dožene jednotlivé země?

In [80]:
data_final = data.merge(data_last_year, left_index=True, right_index=True, suffixes=("_geom", "_ly"))

def estimate_required_time(row, base_column, moving_column, country_count):
  years = (numpy.log(row[moving_column + country_count]) - numpy.log(row[base_column + country_count])) \
         / (row[base_column] - row[moving_column])
  if years < 0:
    return numpy.NAN
  return years

column_list = data_final.columns.tolist()
base_column = column_list.index(f"{BASE_LOCATION}_geom")
for i in range(0, len(LOCATIONS)):
  current_country = column_list[i].replace("_geom", "")
  if current_country == BASE_LOCATION:
    continue
  data_final[f"{current_country}_yrs"] = data_final.apply(estimate_required_time, axis=1, args=(base_column, i, len(LOCATIONS)))
data_final

geo\time,AT_geom,CZ_geom,DE_geom,ES_geom,EU27_2020_geom,AT_ly,CZ_ly,DE_ly,ES_ly,EU27_2020_ly,AT_yrs,DE_yrs,ES_yrs,EU27_2020_yrs
na_item,,,,,,,,,,,,,,
B1GQCP_EUR_HAB,0.026493,0.038706,0.030344,0.015326,0.025779,44780.00,21140.00,41800.0,26420.00,31310.00,61.456408,81.526695,9.536055,30.382615
B1GQCP_PPS_HAB,0.025245,0.038016,0.027099,0.019879,0.026954,40250.00,29730.00,38620.0,28950.00,31990.00,23.722642,23.963995,NaN,6.623288
EARN_NT_NET,0.029807,0.028891,0.022362,0.010648,0.019003,41162.12,15523.25,40947.0,23386.32,28475.11,NaN,148.556057,22.463958,61.357928


## Závěr

- Česká republika v GDP na obyvatele v paritě kupní síly předehnala Španělsko před rokem 2019, Rakousko a Německo by
měla dohnat za 24 let, průměr EU za 7
- V běžných cenách jsou doby podstatně delší
- Příjmy domácností jsou na podstatně nižší úrovni a rostou relativně pomaleji než GDP. Domácnosti tedy bohatly
pomaleji než ekonomika.
- Příjmovou úroveň Španělska by Česká republika měla dohnat až za 22 let, Německa za více než 100 let a Rakousko
v daném období dokonce rostlo rychleji
